In [2]:
import pandas as pd
import numpy as np

In [5]:
companies = pd.read_csv('../data/companies/companies.csv')
industries = pd.read_csv('../data/companies/company_industries.csv')
specialities = pd.read_csv('../data/companies/company_specialities.csv').groupby('company_id').agg({'speciality': lambda x: ', '.join(x)})
employee_counts = pd.read_csv('../data/companies/employee_counts.csv').groupby('company_id').agg(lambda x: int(x.mean()))
company_details = companies.merge(industries, on='company_id')\
                           .merge(specialities, on='company_id')\
                           .merge(employee_counts, on='company_id')\
                           .drop(['address', 'url', 'time_recorded', 'company_size', 'follower_count'], axis=1)
company_details = company_details[company_details['country'] == 'US'][company_details['employee_count'] > 0].sort_values('employee_count', ascending=False)

conditions = [(company_details['employee_count'] <= 100),
              (company_details['employee_count'] > 100) & (company_details['employee_count'] <= 1000),
              (company_details['employee_count'] > 1000) & (company_details['employee_count'] <= 10000),
              (company_details['employee_count'] > 10000)]

sizes = ['early stage startup', 'small', 'medium', 'large']

company_details['size'] = np.select(conditions, sizes)
company_details

/var/folders/v8/l597fdmn691ctljk27sgct540000gn/T/ipykernel_71351/2421040050.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  company_details = company_details[company_details['country'] == 'US'][company_details['employee_count'] > 0].sort_values('employee_count', ascending=False)


,company_id,name,description,state,country,city,zip_code,industry,speciality,employee_count,size
89,1586,Amazon,Amazon is guided by four principles: customer ...,WA,US,Seattle,98109,Software Development,"e-Commerce, Retail, Operations, Internet",748029,large
252,2646,Walmart,"Sixty years ago, Sam Walton started a single m...",Arkansas,US,Bentonville,72712,Retail,"Technology, Transportation, Logistics, Merchan...",429878,large
258,2677,McDonald's,McDonald’s is the world’s leading global foods...,Illinois,US,Chicago,60607,Restaurants,"Restaurant, sustainability, foodservice, franc...",355436,large
109,1680,Cognizant,Cognizant (Nasdaq-100: CTSH) engineers modern ...,New Jersey,US,Teaneck,07666,IT Services and IT Consulting,"Interactive, Intelligent Process Automation, D...",314529,large
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",NY,US,"Armonk, New York",10504,IT Services and IT Consulting,"Cloud, Mobile, Cognitive, Security, Research, ...",312903,large
...,...,...,...,...,...,...,...,...,...,...,...
11910,5219754,Q-SAQ INC,"Q-SAQ, INC is a financially sound US organizat...",Florida,US,MELBOURNE,32940,Wholesale Building Materials,"Vehicle Barriers, Vehicle Detection, Vehicle C...",1,early stage startup
16187,69158025,David Company LLC,David company is a true full-service design an...,Washington,US,Arlington,98223,Architecture and Planning,"Architecture, construction, leasing, commercia...",1,early stage startup
17577,98803167,Blue Horizons Financial LLC,Our mission at Blue Horizons Financial is to c...,0,US,Portland,0,Financial Services,"Retirement Strategies, Child Headstart Plans, ...",1,early stage startup
14353,18008857,International Drivers Academy,The International Drivers Academy is a driving...,Washington,US,Bellevue,98004,Professional Training and Coaching,"State Certified, Driver Training, Defensive Dr...",1,early stage startup


In [6]:
benefits = pd.read_csv('../data/jobs/benefits.csv')
job_industries = pd.read_csv('../data/jobs/job_industries.csv')
job_skills = pd.read_csv('../data/jobs/job_skills.csv')
salaries = pd.read_csv('../data/jobs/salaries.csv')
industries = pd.read_csv('../data/mappings/industries.csv')
skills = pd.read_csv('../data/mappings/skills.csv')

job_details = benefits.merge(job_industries, on='job_id', how='outer')\
                      .merge(job_skills, on='job_id', how='outer')\
                      .merge(skills, on='skill_abr', how='outer')\
                      .merge(industries, on='industry_id', how='outer')\
                      .drop(['skill_abr', 'industry_id', 'inferred'], axis=1)\
                      .astype(str)\
                      .groupby('job_id').agg(lambda x: ', '.join(set(x)))\
                      .merge(salaries.astype(str), on='job_id', how='outer')\
                      .drop(['compensation_type', 'currency', 'salary_id'], axis=1)\
                      .rename({'type': 'benefits',
                               'skill_name':'skills',
                               'industry_name':'industry'}, axis=1)
job_details

,job_id,benefits,skills,industry,max_salary,med_salary,min_salary,pay_period
0,1014822088,401(k),"Sales, Marketing",Software Development,80000.0,nan,70000.0,YEARLY
1,103254301,nan,"Art/Creative, Information Technology, Design",Design Services,300000.0,nan,60000.0,YEARLY
2,103860943,nan,nan,Truck Transportation,19.0,nan,18.0,HOURLY
3,1093227543,nan,"Sales, Product Management, Marketing",Food and Beverage Services,120000.0,nan,60000.0,YEARLY
4,10998357,nan,"Management, Manufacturing",Restaurants,65000.0,nan,45000.0,YEARLY
...,...,...,...,...,...,...,...,...
128422,921716,nan,"Sales, Marketing",Real Estate,20.0,nan,17.0,HOURLY
128423,935210241,nan,Legal,Law Practice,NaN,NaN,NaN,NaN
128424,95428182,"Dental insurance, Medical insurance, Disabilit...",Administrative,Non-profit Organizations,nan,25.0,nan,HOURLY
128425,9615617,nan,nan,Beverage Manufacturing,NaN,NaN,NaN,NaN


In [7]:
postings = pd.read_csv('../data/postings.csv')[['job_id', 
                                             'company_id',
                                            'company_name',
                                            'title', 
                                            'description', 
                                            'location',
                                            'formatted_work_type',
                                            'remote_allowed',
                                            'skills_desc',
                                            'application_url']]
print(f'Postins data available for {len(set(postings["job_id"]))} jobs.')
postings_data = postings.astype(str)\
                        .merge(job_details, on='job_id')

postings_data['remote_allowed'] = postings_data['remote_allowed'].map({'nan': False, '1.0': True})
postings_data['skills_desc'] = postings_data['skills_desc'].astype(str).replace('nan', '')
postings_data['industry'] = postings_data['industry'].astype(str).replace('nan', '')
postings_data['description'] = postings_data['description'] + postings_data['skills_desc']
postings_data.drop(['skills_desc'], axis=1, inplace=True)

postings_data = postings_data[postings_data['application_url'] != "nan"]
print(f'Distinct Application_url available for {len(set(postings["application_url"]))} jobs.')
postings_data

Postins data available for 123849 jobs.
Distinct Application_url available for 84801 jobs.


,job_id,company_id,company_name,title,description,location,formatted_work_type,remote_allowed,application_url,benefits,skills,industry,max_salary,med_salary,min_salary,pay_period
8,1218575,721189.0,Children's Nebraska,Respiratory Therapist,"At Children’s, the region’s only full-service ...","Omaha, NE",Full-time,False,www.childrensnebraska.org,nan,Health Care Provider,Hospitals and Health Care,NaN,NaN,NaN,NaN
16,95428182,55624331.0,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,"Cleveland, OH",Full-time,False,https://www.kidsbookbank.org/employment/,"Dental insurance, Medical insurance, Disabilit...",Administrative,Non-profit Organizations,nan,25.0,nan,HOURLY
37,280496925,166875.0,Washington State University,Coordinator for Multicultural Student Organiza...,The Coordinator serves as the principal adviso...,"Pullman, WA",Full-time,False,https://wsu.wd5.myworkdayjobs.com/en-US/WSU_Jo...,nan,"Training, Education",Higher Education,4817.0,nan,4061.0,MONTHLY
40,368586246,81247311.0,STL Fertility,Embryologist,Job duties:To recover oocytes from follicular ...,"St Louis, MO",Full-time,False,https://www.indeed.com/job/embryologist-944f8c...,"Vision insurance, 401(k)",Health Care Provider,,NaN,NaN,NaN,NaN
46,805229245,nan,nan,"Manager, Retail Pharmacy",SUMMARY:Manages operation and supervises all d...,"Tucson, AZ",Full-time,False,https://jobs.tmcaz.com/manager-pharmacy-retail...,nan,"Sales, Business Development",Hospitals and Health Care,165000.0,nan,110000.0,YEARLY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123706,3906266212,413796.0,Synectics Inc.,Phlebotomist - Float,Job Description\n\nThe Patient Services Repres...,"Carroll County, MD",Contract,False,https://www.synectics.com/candidate-apply.php/...,"Dental insurance, Vision insurance, 401(k)",Science,Staffing and Recruiting,NaN,NaN,NaN,NaN
123707,3906266217,6404239.0,The Dyrt,Senior Frontend/App Developer,The Dyrt is the largest digital camping platfo...,United States,Full-time,True,https://the-dyrt.breezy.hr/p/31c6745b3473-seni...,nan,"Engineering, Information Technology","Technology, Information and Internet",NaN,NaN,NaN,NaN
123708,3906266248,2466850.0,GoodRx,"Account Manager, Client Success",GoodRx is America’s healthcare marketplace. Ea...,United States,Full-time,True,https://goodrx.wd1.myworkdayjobs.com/Careers/j...,"Vision insurance, Medical insurance, 401(k)","Sales, Business Development",Hospitals and Health Care,NaN,NaN,NaN,NaN
123711,3906267126,1124131.0,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,United States,Full-time,True,https://www.pinterestcareers.com/en/jobs/58824...,nan,"Engineering, Information Technology","Technology, Information and Internet, Software...",NaN,NaN,NaN,NaN


In [8]:
match_data = postings_data[['job_id',
                            'company_name',
                            'title',
                            'description',
                            'location',
                            'formatted_work_type',
                            'remote_allowed',
                            'skills',
                            'industry',
                            'application_url']].rename({'formatted_work_type': 'type', 'remote_allowed': 'remote'}, axis=1)
match_data

,job_id,company_name,title,description,location,type,remote,skills,industry,application_url
8,1218575,Children's Nebraska,Respiratory Therapist,"At Children’s, the region’s only full-service ...","Omaha, NE",Full-time,False,Health Care Provider,Hospitals and Health Care,www.childrensnebraska.org
16,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,"Cleveland, OH",Full-time,False,Administrative,Non-profit Organizations,https://www.kidsbookbank.org/employment/
37,280496925,Washington State University,Coordinator for Multicultural Student Organiza...,The Coordinator serves as the principal adviso...,"Pullman, WA",Full-time,False,"Training, Education",Higher Education,https://wsu.wd5.myworkdayjobs.com/en-US/WSU_Jo...
40,368586246,STL Fertility,Embryologist,Job duties:To recover oocytes from follicular ...,"St Louis, MO",Full-time,False,Health Care Provider,,https://www.indeed.com/job/embryologist-944f8c...
46,805229245,nan,"Manager, Retail Pharmacy",SUMMARY:Manages operation and supervises all d...,"Tucson, AZ",Full-time,False,"Sales, Business Development",Hospitals and Health Care,https://jobs.tmcaz.com/manager-pharmacy-retail...
...,...,...,...,...,...,...,...,...,...,...
123706,3906266212,Synectics Inc.,Phlebotomist - Float,Job Description\n\nThe Patient Services Repres...,"Carroll County, MD",Contract,False,Science,Staffing and Recruiting,https://www.synectics.com/candidate-apply.php/...
123707,3906266217,The Dyrt,Senior Frontend/App Developer,The Dyrt is the largest digital camping platfo...,United States,Full-time,True,"Engineering, Information Technology","Technology, Information and Internet",https://the-dyrt.breezy.hr/p/31c6745b3473-seni...
123708,3906266248,GoodRx,"Account Manager, Client Success",GoodRx is America’s healthcare marketplace. Ea...,United States,Full-time,True,"Sales, Business Development",Hospitals and Health Care,https://goodrx.wd1.myworkdayjobs.com/Careers/j...
123711,3906267126,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,United States,Full-time,True,"Engineering, Information Technology","Technology, Information and Internet, Software...",https://www.pinterestcareers.com/en/jobs/58824...


In [10]:
import ollama
import pickle

states_detected = []

for location in match_data.location:
    prompt = f'''
    This is a location in the US: {location}. Return the full name of the State it is located in. If no particular state is found, return an empty string. Do not explain, just return the name of the state.
    '''
    states_detected.append(ollama.generate(model = "llama3", prompt = prompt)['response'])

match_data['location'] = states_detected

with open('../data/states.pkl', 'rb') as file:
    states = pickle.load(file)

match_data.loc[~match_data['location'].isin(states), 'location'] = ''

match_data.to_json('../data/jobs.json', orient='records')